Extract feature vectors from text corpus using universal-sentence-encoder-multilingual

Author: Christopher Pietsch

In [5]:
!pip3 install tensorflow_text>=2.0.0rc0

In [6]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

import pandas as pd
import altair as alt

import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
# The 16-language multilingual module is the default but feel free
# to pick others from the list and compare the results.
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']

model = hub.load(module_url)

In [8]:
data = pd.read_csv("/content/export1305.csv")

In [9]:
titles = data["titel"].fillna("")
titles_embeds = model(titles)

In [11]:
titles_embeds

<tf.Tensor: shape=(5142, 512), dtype=float32, numpy=
array([[-0.00405282,  0.00879698,  0.0687516 , ..., -0.08264694,
         0.04315777,  0.01709835],
       [ 0.00521728,  0.00492668,  0.00010258, ...,  0.00226365,
        -0.0688452 ,  0.0221793 ],
       [-0.02151257, -0.03914239, -0.04092657, ...,  0.02300562,
        -0.03889342,  0.00444949],
       ...,
       [ 0.05813951, -0.00295822, -0.04776495, ...,  0.00208871,
         0.02377026,  0.00749476],
       [-0.02196946,  0.0025272 , -0.01683729, ..., -0.03915028,
        -0.03435468, -0.08900943],
       [-0.02689372, -0.05531327, -0.03047121, ..., -0.03103877,
        -0.01666632, -0.0520621 ]], dtype=float32)>

In [10]:
np.inner(titles_embeds, titles_embeds)

array([[ 0.9999999 ,  0.20228577,  0.18218866, ...,  0.0160306 ,
         0.11675507,  0.14250843],
       [ 0.20228577,  1.0000001 , -0.01690983, ...,  0.03832811,
         0.06965499, -0.01276348],
       [ 0.18218866, -0.01690983,  0.9999998 , ...,  0.04409757,
         0.28415787,  0.1498137 ],
       ...,
       [ 0.0160306 ,  0.03832811,  0.04409757, ...,  1.        ,
         0.2112321 ,  0.19714086],
       [ 0.11675507,  0.06965499,  0.28415787, ...,  0.2112321 ,
         0.99999994,  0.3634357 ],
       [ 0.14250843, -0.01276348,  0.1498137 , ...,  0.19714086,
         0.3634357 ,  1.0000001 ]], dtype=float32)

In [13]:
def arr_to_csv(arr, filename, mult=10000):
  dataframe = (pd.DataFrame(arr) * mult).round(0).astype('int32')
  dataframe['id'] = data['id']
  dataframe.to_csv(filename)
  return dataframe

In [14]:
dataframe = arr_to_csv(titles_embeds, "use.csv")
dataframe

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,id
0,-40,87,687,-244,653,-233,-167,-775,-796,288,312,654,251,441,-209,-6,-9,248,-477,332,-230,153,-345,244,328,-180,-317,-234,-758,-11,-181,-268,-394,-588,814,385,-881,-274,332,112,...,379,804,771,-3,-901,640,45,-131,-559,-462,-142,234,214,236,742,339,232,276,166,-117,37,345,422,331,597,-772,-790,157,546,253,160,-326,169,-503,681,-184,-826,431,170,968378
1,52,49,1,-531,794,-740,214,-645,1001,-98,120,-566,76,-136,-38,-109,207,97,247,-110,-574,492,-75,12,-169,-334,-15,367,-340,119,-16,-552,58,-54,1225,-15,-191,35,-925,656,...,-754,-479,544,-172,-200,572,-93,175,-23,-489,-371,-478,-381,333,986,-506,6,-621,116,-307,73,106,-343,-480,-193,338,-823,11,-484,-438,-222,-350,-424,462,814,359,22,-688,221,959351
2,-215,-391,-409,-349,138,1142,686,292,-898,39,120,409,-390,893,-367,145,391,-760,201,360,-130,421,146,541,-456,-471,-441,-493,-54,-162,323,-148,118,-467,-7,797,-372,469,651,-771,...,30,-257,717,-118,-412,574,763,-134,281,571,241,99,211,-371,-55,272,37,-115,370,-110,255,623,-601,470,479,-109,-728,577,16,-118,652,153,786,-176,149,-825,230,-388,44,962134
3,-136,-499,-582,-455,494,98,-673,-374,-568,-384,-85,-264,-32,-148,-172,-292,-658,395,-314,400,-693,845,459,251,466,72,-614,-236,-735,-240,-619,154,204,-261,-34,-2,-491,755,806,674,...,-15,-50,-595,79,-496,-77,616,-327,-555,993,92,-221,334,-270,-146,149,599,582,-179,-221,-851,91,255,511,-353,-278,-420,-79,179,36,-95,234,4,-806,659,247,68,-168,29,961042
4,-130,-156,10,140,917,-123,625,-428,125,95,-244,229,240,159,-118,-894,-201,281,558,402,-727,163,163,223,166,591,-519,-1114,-896,34,35,67,-827,-432,-38,205,-780,-319,-118,-310,...,-41,312,-262,272,187,499,303,56,171,-190,355,-237,-642,172,498,558,-66,-348,-290,-139,306,-81,347,609,92,188,-866,633,-117,-589,-233,-127,-245,-485,256,-223,-144,-131,-363,963307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5137,-282,306,792,-220,803,60,-426,-297,372,273,324,142,280,-499,-649,-658,-435,233,179,60,-134,-169,-1316,293,1015,380,139,-425,67,-381,-100,581,-268,-93,382,-363,119,432,-192,-123,...,474,-306,166,-986,-143,-243,886,254,-249,-883,-444,264,362,-825,761,407,-582,44,477,-427,-6,-18,171,-576,44,34,-157,-705,-124,828,-735,235,351,484,226,-1213,275,100,91,567563
5138,-397,247,763,-194,839,-220,-561,-763,222,106,-416,-418,203,-675,-145,-159,20,77,62,104,1169,76,-467,113,633,-86,-250,49,442,213,-280,-109,-148,9,231,805,124,339,-600,601,...,237,-351,-145,-766,128,-78,-59,998,-768,-851,405,22,-296,214,638,-137,921,-446,-78,-415,-8,-359,66,-513,-104,207,-311,334,-377,365,74,1008,263,134,419,-265,-800,-265,618,1486490
5139,581,-29,-477,172,-730,260,-950,-217,-251,-287,415,-95,-847,506,290,-329,-599,-287,568,-327,47,-340,-178,569,-94,793,-276,-90,-513,-171,208,206,-148,78,14,55,313,-257,-93,-852,...,-341,318,-10,94,-246,-256,408,303,-198,972,115,215,-100,-492,131,-96,624,777,437,-133,47,-941,-17,273,521,430,-26,531,115,-103,684,-326,-422,-485,311,-420,20,237,74,1532768
5140,-219,25,-168,77,-705,259,276,450,-763,-161,-668,-125,-263,595,-12,202,-772,-14,-747,-368,400,143,175,268,-513,-282,-387,-763,266,-326,-95,120,-22,-367,-524,285,-648,119,11,-247,...,314,-62,695,-444,-35,-76,-183,-783,130,979,-25,-540,-257,194,883,-24,2,-127,195,87,206,-963,-389,203,283,199,-515,413,186,-71,-51,446,-250,-110,247,-355,-391,-343,-890,2294475


In [17]:
import umap

def run_umap(data):
  umap_op = umap.UMAP(n_components=2)
  data_umap = umap_op.fit_transform(data)
  data_umap_df = pd.DataFrame(data=data_umap, columns=['x', 'y'])
  return data_umap_df

In [18]:
umap_title = run_umap(titles_embeds)

In [20]:
alt.data_transformers.disable_max_rows()
alt.Chart(pd.concat([titles,umap_title], axis=1)).encode(
      x='x',
      y='y',
      tooltip=['titel']
  ).mark_circle(size=60).properties(
      width=500, height=500
  ).interactive()

alt.Chart(...)